In [1]:
import pandas as pd
import numpy as np
import tflearn
from tflearn.data_utils import to_categorical

C:\Users\Owner\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Owner\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Owner\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Owner\AppData\Roaming\Python\Python36\site-

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [2]:
df_reviews = pd.read_csv('reviews.txt', header=None)
df_labels = pd.read_csv('labels.txt', header=None)

In [3]:
words = [word for reviews in df_reviews.to_numpy() for review in reviews for word in review.split()]
len(words)

6347388

In [5]:
from collections import Counter
c = Counter(words)
c['high']

2161

In [ ]:
reviews_values = [arr_reviews_values for arr_reviews_values in df_reviews.to_numpy()]

In [ ]:
reviews_values[0]

In [ ]:
reviews = [review[0] for review in reviews_values]

In [ ]:
reviews[0]

In [ ]:
reviews[0].split()

In [ ]:
words = []
for review in reviews:
    a = review
    for word in a.split():
        words += [word]


In [ ]:
words = [word for reviews in df_reviews.to_numpy() for review in reviews for word in review.split()]
len(words)


In [ ]:
words = [word for reviews in df_reviews.to_numpy() for review in reviews for word in review.split()]
len(words)


In [ ]:
words[2]

In [ ]:
review = reviews[1]
print (review)
for word in review.split(' '):
    words += word
    
words

In [ ]:
from collections import Counter
c = Counter(words)

In [ ]:
c['bromwell']
    

In [ ]:
words = [word for word in [review.split() for review in reviews]]
for review in reviews:
    for word in review.split():
        words += word
        
words = [word for word in [review.split() for review in [arr_review_value[0] for arr_review_value in 

In [ ]:
i = 0
for row in df_reviews.itertuples():
    print(row)
    print("---")
    i += 1
    if i > 3:
        break

In [ ]:
from collections import Counter

r = reviews.stack().values

In [ ]:
r = reviews.values
r[1][0].split()

In [ ]:
reviews = [review for review in reviews.values]

In [ ]:
review[1]

In [ ]:
df_review_values = [df_review_values for df_review_values in df_reviews.values[0]]

In [ ]:
words = [words for review in reviews [for df_reviews in df_reviews.values[0]]
len(words)

In [ ]:
len(b)

In [ ]:
# reviews = [review for review in df_reviews.values[0]]
#reviews[0]
#for review in reviews:
#    for word in review.split()
    
words = [word for word in [review.split() for review in [df_review_value[0] for df_review_value in df_reviews.values]]]

In [ ]:
words[1]

In [ ]:
words = [word for word in [review in df_reviews.values] ]


In [ ]:
a = "the quick brown fox jumps of the lazy dog"
words = [word for word in a.split()]
print (words)

In [ ]:
words = a.split()
print (words)

In [ ]:
a